In [76]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import time
import pandas as pd
import requests

In [77]:
# 크롬 웹드라이버 실행 경로
path = 'chromedriver.exe'

# 크롤링할 주소 (네이버 세부설정까지 완료한 상태로)
target_url = "https://search.naver.com/search.naver?where=news&query=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90&sm=tab_opt&sort=2&photo=0&field=0&pd=3&ds=2023.07.01&de=2023.12.31&docid=&related=0&mynews=1&office_type=1&office_section_code=3&news_office_checked=1009&nso=so%3Ar%2Cp%3Afrom20230701to20231231&is_sug_officeid=0&office_category=0&service_area=0"

# 크롬 드라이버 사용
service = Service(executable_path=path) # selenium 최근 버전은 이렇게
options = webdriver.ChromeOptions()
driver = webdriver.Chrome()

# 드라이버에게 크롤링할 대상 알려주기
try:
    # 웹페이지 열기
    driver.get(target_url)

    # 초기 스크롤 높이 설정
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # 아래까지 스크롤 다운
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # 적응 시간
        time.sleep(0.5)  

        # 새로운 스크롤 높이 계산
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break  # 만약 더 스크롤할게 없다면 정지
        last_height = new_height

    # 해당 화면 안에서 페이지 소스 받기
    page_source = driver.page_source

    # BeautifulSoup으로 parsing
    bs_obj = BeautifulSoup(page_source, 'html.parser')

    # a태그로 info에 해당하는 태그들 다 수집
    results = bs_obj.find_all('a', {'class': "info"})

    # 수집한 a 태그 개수 -> 절반이 네이버 뉴스
    print(f"Number of results found: {len(results)/2}")

finally:
    # 가상 화면 닫기
    driver.quit()

Number of results found: 2150.0


In [78]:
# 기존 한국경제 링크랑 섞여서 네이버 뉴스꺼만 필터링
news_link = []
for tag in results:
    href = tag.get('href')
    if href:
        news_link.append(href)

news_link = [i for index, i in enumerate(news_link) if (index + 1) % 2 == 0]

In [79]:
driver = webdriver.Chrome()
driver.implicitly_wait(3)

In [80]:
from tqdm import tqdm

all_results = dict()

# tqdm을 사용하여 진행 상황 표시
for i, link in enumerate(tqdm(news_link, desc="Processing links")):     
    # scraping하려는 웹페이지 주소를 get()에 전달
    driver.get(link)
    all_results[i] = dict()
    
    # 일시 추출하기(연예, 스포츠 예외처리)
    try:
        date = driver.find_elements(By.CLASS_NAME, 'media_end_head_info_datestamp_bunch')
        date = date[0].text[2:]
    except (IndexError, AttributeError):
        try:
        # 첫 번째 시도가 실패했을 경우, 두 번째 클래스로 시도
            date = driver.find_elements(By.CLASS_NAME, 'date')
            date = date[0].text[2:]
        except (IndexError, AttributeError):
            try:
            # 두 번째 시도가 실패했을 경우, 세 번째 클래스로 시도
                date = driver.find_elements(By.CLASS_NAME, 'NewsEndMain_date__xjtsQ')
                date = date[0].text[2:]
            except (IndexError, AttributeError):
            # 세 번째 시도마저 실패했을 경우 "사라진 페이지" 출력
                date = "사라진 페이지"
    
    # 제목 추출하기(연예, 스포츠 예외처리)
    try:
        title = driver.find_elements(By.CLASS_NAME, 'media_end_head_headline')
        title = title[0].text
    except (IndexError, AttributeError):
        try:
            title = driver.find_elements(By.CLASS_NAME, 'NewsEndMain_article_head_title__ztaL4')
            title = title[0].text
        except (IndexError, AttributeError):
        # 두 번째 시도마저 실패했을 경우 "사라진 페이지" 출력
            title = "사라진 페이지"
        
    # 본문 추출하기(연예, 스포츠 예외처리)
    try:
        body = driver.find_elements(By.ID, 'newsct_article')
        body = body[0].text.replace('\n','')
    except (IndexError, AttributeError):
        try:
            body = driver.find_elements(By.CLASS_NAME, '_article_content')
            body = body[0].text.replace('\n','')
        except (IndexError, AttributeError):
        # 두 번째 시도마저 실패했을 경우 "사라진 페이지" 출력
            body = "사라진 페이지"
    
    # 결과 저장
    all_results[i]['link'] = link
    all_results[i]['date'] = date
    all_results[i]['title'] = title
    all_results[i]['body'] = body

# 드라이버 종료
driver.quit()

Processing links: 100%|██████████████████████████████████████████████████████████| 2150/2150 [1:06:54<00:00,  1.87s/it]


In [81]:
news_df = pd.DataFrame(all_results).T

In [82]:
news_df = news_df[news_df['date'] != '사라진 페이지']

In [83]:
# '오전'과 '오후'를 영어로 변환
news_df['date'] = news_df['date'].str.replace('오전', 'AM').str.replace('오후', 'PM')

# date 열을 datetime 형식으로 변환
def try_convert_date(x):
    try:
        # 첫 번째 시도: 원래 문자열을 포맷에 맞게 변환
        return pd.to_datetime(x, format='%Y.%m.%d. %p %I:%M')
    except ValueError:
        # 첫 번째 시도가 실패할 경우: 앞에 "20"을 추가하여 다시 변환 시도
        return pd.to_datetime("20" + x, format='%Y.%m.%d. %p %I:%M')

news_df['date'] = news_df['date'].apply(try_convert_date)
                                            
# year, month, day, hour, minute 열 생성
news_df['year'] = news_df['date'].dt.year
news_df['month'] = news_df['date'].dt.month
news_df['day'] = news_df['date'].dt.day
news_df['hour'] = news_df['date'].dt.hour
news_df['minute'] = news_df['date'].dt.minute

In [84]:
# 고유 인덱스 부여 (년도 + 6자리)
# 1이면 -> 000001
news_df.index = news_df.index + 1
news_df = news_df.reset_index()
news_df['index'] = news_df['index'].astype(str).str.zfill(6)
news_df['index'] = str(news_df['year'].unique()[0]) + news_df['index']

In [85]:
news_df

,index,link,date,title,body,year,month,day,hour,minute
0,2023000001,https://n.news.naver.com/mnews/article/009/000...,2023-07-01 08:57:00,"반나절도 못 가는 스마트폰 배터리, 쉽게 교체가 가능해진다고요? [뉴스 쉽게보기]","애플의 아이폰14 [사진=애플]스마트폰 배터리 전원이 부족해 곤란했던 상황, 한 번...",2023,7,1,8,57
1,2023000002,https://n.news.naver.com/mnews/article/009/000...,2023-07-01 14:18:00,미국 이 기업 실적 발표됐는데 왜 뜬금없이 삼성전자가 ‘신고가’? [MK위클리반도체],지난 29일 장중 삼성전자가 52주 신고가를 달성하면서 ‘8만 전자’ 복귀라는 투자...,2023,7,1,14,18
2,2023000003,https://n.news.naver.com/mnews/article/009/000...,2023-07-02 07:07:00,"이재용 삼행시에 열광하는 팬들, 문제는 가짜란거야 [방영덕의 디테일]",이재용 삼성전자 회장 팬페이지를 자처하는 계정(@jaeyoung3831)에 올라온 ...,2023,7,2,7,7
3,2023000004,https://n.news.naver.com/mnews/article/009/000...,2023-07-02 14:07:00,올 여름 이거 자주 했다간 전기료 폭탄?…에어컨 절전 이렇게 하세요,[사진출처= 연합뉴스]올 여름 전기료 인상으로 에어컨을 켜기가 겁난다는 가정이 많다...,2023,7,2,14,7
4,2023000005,https://n.news.naver.com/mnews/article/009/000...,2023-07-02 16:09:00,재테크 공부 '1000원'으로 시작해보세요,MZ세대라면 종잣돈 모으기만큼 중요한 것이 '투자 마인드'를 갖추는 것이다. 통상 ...,2023,7,2,16,9
...,...,...,...,...,...,...,...,...,...,...
2144,2023002146,https://n.news.naver.com/mnews/article/009/000...,2023-12-31 13:14:00,“이 칼라 실화야? 가격도 착해진다고?”…갤럭시 S24 예상 스펙 총정리,갤럭시S24 시리즈 렌더링 이미지. [사진출처=폰아레나]삼성전자의 첫 인공지능(AI...,2023,12,31,13,14
2145,2023002147,https://n.news.naver.com/mnews/article/009/000...,2023-12-31 13:47:00,알트만·나델라…AI 스타들이 스위스로 가는 까닭 [지식人 지식in],다보스포럼이 열리는 스위스 다보스의 1월 풍경세계 최대 규모의 글로벌 포럼인 다보스...,2023,12,31,13,47
2146,2023002148,https://n.news.naver.com/mnews/article/009/000...,2023-12-31 14:34:00,2차전지 급등에 그룹 시총 순위 뒤집혔다…포스코 5위·에코프로 6위로,2차전지 관련기업 시총 급증포스코 5위·에코프로 6위로인터넷 부진에 네카오 순위 하...,2023,12,31,14,34
2147,2023002149,https://n.news.naver.com/mnews/article/009/000...,2023-12-31 17:23:00,올해 글로벌 최저한세 시행…국내 기업 300곳 직격탄,세율 15% 미만 국가서 사업땐법인세 더 내 이익 감소 우려헝가리 진출 기업 등 영...,2023,12,31,17,23


In [86]:
news_df.to_csv('2023_매일경제_후반기.csv', index=False, encoding='utf-8-sig')